# 在 Llama Index 中使用千帆能力

LlamaIndex是一个用于连接大语言模型（LLMs）和外部数据源的数据框架。它能够让LLMs访问和利用私有或领域特定的数据，以优化模型性能并使其更加易用和流畅。

本文准备了一份在 Llama Index 中使用千帆的能力进行 RAG 的示例，供用户参考。

# 前置准备

用户首先需要自行安装集成了 OpenAI 组件的 Llama Index，以及安装最新版本的千帆 Python SDK

In [ ]:
!pip install -U "qianfan[openai]"
!pip install -U llama-index
!pip install -U nest_asyncio

在安装完成之后，我们还需要在 Shell 中设置相关环境变量并启动千帆的 OpenAI 转接服务。

这个服务负责监听本地发出的、指向 OpenAI 的请求，并将请求映射并转发到千帆，最后从千帆拿回结果。

从而实现使得各类适配了 OpenAI 服务的框架，能够无感、零代码接入千帆平台。

In [1]:
import nest_asyncio
import os
import subprocess as sp

nest_asyncio.apply()

# 该环境变量用于绕过 LlamaIndex 对 OpenAI API Key 的检查
os.environ["OPENAI_API_KEY"] = "FAKE_KEY"

# 需要将 OPENAI 的 API 请求重定向到本地的服务
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8001/v1"

# 以下环境变量供千帆 OpenAI Adapter 使用
os.environ["QIANFAN_ACCESS_KEY"] = "your_qianfan_console_access_key"
os.environ["QIANFAN_SECRET_KEY"] = "your_qianfan_console_secret_key"
os.environ["QIANFAN_QPS_LIMIT"] = "5"

# 启动千帆 SDK 的 OpenAI Adapter 服务，用于代理转发指向 OpenAI 的请求，并替换为千帆服务的返回
server = sp.Popen("qianfan openai -p 8001", shell=True)

OpenAI wrapper server is running at                                             

 • http://127.0.0.1:8001                                                        
 • http://172.18.183.188:8001                                                   

Remember to set the environment variables:                                      

                                                                                
     export OPENAI_API_KEY='any-content-you-want'                               
     export OPENAI_BASE_URL='http://172.18.183.188:8001/v1'                     
                                                                                



INFO:     Started server process [44989]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


# 加载数据与构建向量库

Llama Index 提供的数据加载功能允许我们直接从文件夹方便的构建向量数据库

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter

transformations=[
    SentenceSplitter(chunk_size=50, chunk_overlap=0),
    # 这里虽然使用的是 OpenAI 的 Embedding 服务组件
    # 但是实际上发出去的请求会被重定向到千帆
    # 由千帆进行实际的处理操作
    # 目前实际使用的千帆模型为 Embeddings-V1
    # 且只支持使用 Embeddings-V1 进行 Embedding
    OpenAIEmbedding(embed_batch_size=10),
]

index = VectorStoreIndex.from_documents(documents, transformations=transformations, show_progress=True)


INFO:     127.0.0.1:58923 - "POST /v1/embeddings HTTP/1.1" 200 OK



Generating embeddings: 0it [00:00, ?it/s]


# 查询文本

在构建完成后，`index` 对象就是一个包含了上面文本的向量化表示的向量数据库。我们可以直接使用该向量数据库所提供的大模型查询方法 `query` ，用自然语言来查询向量数据库中的文本。

In [3]:
# 这里支持传入OPEN AI 的模型名。相关的模型名会被映射到千帆的模型名。
query_engine = index.as_query_engine()

# 如果用户安装了 Langchain，还可以直接使用千帆在 Langchain 中的 ChatModel 组件
# 来在 LlamaIndex 中直接使用千帆的大模型服务
# 如下所示，使用前请解除注释
# from langchain.chat_models.baidu_qianfan_endpoint import QianfanChatEndpoint
# query_engine = index.as_chat_engine(llm=QianfanChatEndpoint())

response = query_engine.query("What did the author do growing up?")
print(f"\n大模型回答: {response}\n")

[INFO] [03-28 18:50:29] openapi_requestor.py:377 [t:8094817088]: async requesting llm api endpoint: /embeddings/embedding-v1


INFO:     127.0.0.1:59061 - "POST /v1/embeddings HTTP/1.1" 200 OK


[INFO] [03-28 18:50:30] openapi_requestor.py:377 [t:8094817088]: async requesting llm api endpoint: /chat/completions


INFO:     127.0.0.1:59063 - "POST /v1/chat/completions HTTP/1.1" 200 OK

大模型回答: Based on the provided context information, there is no mention of what the author did growing up. The context only provides information about a file path and excerpts from a text file named "paul_graham_essay.txt". Therefore, it is not possible to answer the query based on the given context.



最后关闭我们启动的服务

In [4]:
server.terminate()

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [44989]


: 